In [ ]:
import sys
import copy
import math
import pickle

sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

In [ ]:
def sample(pts_silent, pts_implicit1, R, C, sigma):
    # We first pick a random index in pts_silent and get the normal position
    # We substract 1 to the possible index so we can easily get the tangent vector
    # ind = np.asscalar(np.random.choice(pts_silent.shape[0] - 1, 1))
    # tan = (pts_silent[ind + 1] - pts_silent[ind]).numpy()
    # norm = np.array([tan[1], -tan[0]])
    # norm /= np.linalg.norm(norm)
    # norm *= -150.
    # cotan = torch.zeros_like(pts_silent)
    # cotan[ind] = torch.tensor(norm)

    cotan = 20.*torch.randn_like(pts_silent)
    
    silent = dm.DeformationModules.SilentLandmarks.build_from_points(pts_silent)
    silent.manifold.fill_cotan(cotan.view(-1).requires_grad_())
    implicit1 = dm.DeformationModules.ImplicitModule1.build_and_fill(2, pts_implicit1.shape[0], C, sigma, 0.001, gd=(pts_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_()))

    dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 20, 'midpoint')

    return silent.manifold.gd.detach().view(-1, 2)


In [ ]:
peanut = torch.tensor(pickle.load(open("../../data/peanuts.pickle", 'rb'))[0][0]).type(torch.get_default_dtype())

aabb_peanut = dm.Utilities.AABB.build_from_points(peanut)

density = 30

pts_implicit1 = dm.Utilities.fill_aabb(aabb_peanut, density)

print(aabb_peanut.area*density)
print(pts_implicit1.shape[0])

In [ ]:
plt.plot(peanut.numpy()[:, 0], peanut.numpy()[:, 1])
plt.plot(pts_implicit1.numpy()[:, 0], pts_implicit1.numpy()[:, 1], '.')
plt.axis('equal')
plt.show()

In [ ]:
C = torch.zeros(pts_implicit1.shape[0], 2, 2)
C[pts_implicit1[:, 0] >= 0, 0, 0] = -10.
C[pts_implicit1[:, 0] >= 0, 1, 0] = 5.
C[pts_implicit1[:, 0] < 0, 0, 1] = 20.
C[pts_implicit1[:, 0] < 0, 1, 1] = 8.

%matplotlib qt5
ax = plt.subplot()
dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, c_index=0, scale=0.01, color='red')
dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, c_index=1, scale=0.01, color='blue')
dm.Utilities.plot_C_arrow(ax, pts_implicit1, C, c_index=0, scale=0.01, color='black', zorder=3, mutation_scale=10)
dm.Utilities.plot_C_arrow(ax, pts_implicit1, C, c_index=1, scale=0.01, color='black', zorder=3, mutation_scale=10)
plt.axis(aabb_peanut.get_list())
plt.axis('equal')
plt.show()

In [ ]:
angles = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(a) for a in angles])

my_sample = sample(peanut, pts_implicit1, R, C, 0.5)

ax = plt.subplot()
plt.plot(peanut[:, 0].numpy(), peanut[:, 1].numpy(), '--')
plt.plot(my_sample[:, 0].numpy(), my_sample[:, 1].numpy())
plt.axis('equal')
plt.show()

In [ ]:
sample_count = 40
samples = [peanut.tolist()]
for i in range(sample_count):
    print("Generating sample", i)
    samples.append(sample(peanut, pts_implicit1, R, C, 0.4).tolist())

In [ ]:
# Does not work somehow

nb_plot_row = 5
nb_plot_col = 5
cur_plot_index = 0
while True:
    for i in range(nb_plot_row):
        for j in range(nb_plot_col):
            if cur_plot_index < len(samples):
                plt.subplot(nb_plot_row, nb_plot_col, nb_plot_row*i+j+1)
                plt.plot(np.asarray(samples[cur_plot_index])[:, 0], np.asarray(samples[cur_plot_index])[:, 1])
                cur_plot_index = cur_plot_index + 1
            else:
                plt.show()
                break

    plt.show()        

In [ ]:
pickle.dump(samples, open("../../data/deformed_peanuts.pkl", 'wb'))